**DEEP LEARNING**

RBM-Tensorflow

GABRIEL JOSHUA . R


### Importing necessary libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### Loading MNIST dataset

In [ ]:
(train_data, _), (test_data, _) = tf.keras.datasets.mnist.load_data()
train_data = train_data / np.float32(255)
train_data = np.reshape(train_data, (train_data.shape[0], 784))

In [ ]:
test_data = test_data / np.float32(255)
test_data = np.reshape(test_data, (test_data.shape[0], 784))

### Define Restricted Boltzmann Machine (RBM) class

In [ ]:
class RBM(object):
    def __init__(self, input_size, output_size, lr=1.0, batchsize=100):
        """
        Initialize RBM with hyperparameters.
        """
        self._input_size = input_size  # Size of visible layer
        self._output_size = output_size  # Size of hidden layer
        self.learning_rate = lr  # The step used in gradient descent
        self.batchsize = batchsize  # The size of the training data used per sub-iteration

        # Initialize weights and biases as matrices full of zeroes
        self.w = tf.zeros([input_size, output_size], np.float32)  # Weight matrix
        self.hb = tf.zeros([output_size], np.float32)  # Hidden biases
        self.vb = tf.zeros([input_size], np.float32)  # Visible biases

    # Forward Pass: Probability of hidden units given visible units
    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    # Backward Pass: Probability of visible units given hidden units
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)

    # Generate the sample probability for binary units
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    # Training method for the model
    def train(self, X, epochs=10):
        loss = []
        for epoch in range(epochs):
            # For each step/batch
            for start, end in zip(range(0, len(X), self.batchsize), range(self.batchsize, len(X), self.batchsize)):
                batch = X[start:end]

                # Gibbs sampling: Contrastive Divergence
                h0 = self.sample_prob(self.prob_h_given_v(batch, self.w, self.hb))  # p(h|input)
                v1 = self.sample_prob(self.prob_v_given_h(h0, self.w, self.vb))  # p(v|h)
                h1 = self.prob_h_given_v(v1, self.w, self.hb)  # p(h|v)

                # Create the Gradients
                positive_grad = tf.matmul(tf.transpose(batch), h0)
                negative_grad = tf.matmul(tf.transpose(v1), h1)

                # Update weights and biases using gradient descent
                self.w = self.w + self.learning_rate * (positive_grad - negative_grad) / tf.dtypes.cast(tf.shape(batch)[0], tf.float32)
                self.vb = self.vb + self.learning_rate * tf.reduce_mean(batch - v1, 0)
                self.hb = self.hb + self.learning_rate * tf.reduce_mean(h0 - h1, 0)

            # Calculate reconstruction error and store it
            err = tf.reduce_mean(tf.square(batch - v1))
            print('Epoch: %d' % epoch, 'reconstruction error: %f' % err)
            loss.append(err)

        return loss

    # Generate output from the RBM
    def rbm_output(self, X):
        out = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        return out

    # Reconstruct input data using the RBM
    def rbm_reconstruct(self, X):
        h = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        reconstruct = tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.w)) + self.vb)
        return reconstruct

### Instantiate the RBM class

In [ ]:
input_size = train_data.shape[1]
rbm = RBM(input_size, 200)

### Train the RBM model

In [ ]:
err = rbm.train(train_data, 50)

### Plot the reconstruction error over epochs

In [ ]:
plt.plot(err)
plt.xlabel('epochs')
plt.ylabel('cost')

### Reconstruct test data using the trained RBM model

In [ ]:
out = rbm.rbm_reconstruct(test_data)

### Plot original and reconstructed images side by side


In [ ]:
row, col = 2, 8
idx = np.random.randint(0, 100, row * col // 2)
f, axarr = plt.subplots(row, col, sharex=True, sharey=True, figsize=(20, 4))
for fig, row in zip([test_data, out], axarr):
    for i, ax in zip(idx, row):
        ax.imshow(tf.reshape(fig[i], [28, 28]), cmap='Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
